In [ ]:
from itertools import combinations
from collections import defaultdict, namedtuple
from math import isclose

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.patches import Polygon

from PIL import *

rng = np.random.default_rng(seed=1)

grid_count = 5
grid_bounds = [-3, 3]
grid_bounds[1] += 1
#cos_list = [np.cos(x * np.pi / grid_count) for x in np.arange(grid_count)]
#sin_list = [np.sin(x * np.pi / grid_count) for x in np.arange(grid_count)]
cos_list = [np.cos(x * 2 * np.pi / grid_count) for x in np.arange(grid_count)]
sin_list = [np.sin(x * 2 * np.pi / grid_count) for x in np.arange(grid_count)]

#offsets = rng.uniform(size=grid_count)
#offsets = offsets / np.sum(offsets)
offsets = np.repeat(1/grid_count, grid_count)
#offsets = np.repeat(.2, grid_count)
# offsets = np.empty((grid_count,),float)
# offsets[::2] = (1/grid_count)
# offsets[1::2] = -(1/grid_count)
#offsets = np.repeat(0, grid_count)
#offsets = np.repeat(1/grid_count, grid_count)

base_point = np.array([5, 5], dtype=float)

def determine_base_K(p, c, s, o):
    # If it's on a line, lower it to the origin
    if isclose(((c * p[0] + s * p[1]) % 1), o, rel_tol=1e-5, abs_tol=1e-05):
        p_distance = np.sqrt(p[0]**2 + p[1]**2)
        p_normal_origin = [-p_ / p_distance * .5 for p_ in p]
        p = [p + pno for p, pno in zip(p, p_normal_origin)]

    base_k = np.ceil(c * p[0] + s * p[1] - o)
    return base_k

base_k_values = []
for c, s, o in zip(cos_list, sin_list, offsets):
    base_k_values.append(determine_base_K(base_point, c, s, o))

mid_point = np.array([0, 0], dtype=float)
for k, c, s in zip(base_k_values, cos_list, sin_list):
    mid_point += k * np.array([c, s])

class Line:
    # a*x + b*y = c
    def __init__(self, a, b, o, k, grid):
        self.a = a
        self.b = b
        self.c = o + k
        self.offset = o
        self.k = k
        self.grid = grid

        self.first_angle = np.arctan2(-a, b)
        if self.first_angle < 0:
            self.second_angle = self.first_angle + np.pi
        else:
            self.second_angle = self.first_angle - np.pi

        # crossing counter clockwise, first angle is k + 1, second is k
        # because of how np.arctan2 works?
        Angle = namedtuple('Angle', ['value', 'k', 'grid'])

        self.angles = [Angle(self.first_angle, self.k + 1, grid), 
                       Angle(self.second_angle, self.k, grid)]

        # Find closes point to the origin
        self.origin_x = (self.a * self.c) / (self.a**2 + self.b**2)
        self.origin_y = (self.b * self.c) / (self.a**2 + self.b**2)

        self.perpendicular_angle = np.arctan2(b, a)

        # Keep this stuff in here for testing, but add parameter to exclude it later
        self.p1 = [np.cos(self.perpendicular_angle) * .25 + self.origin_x,
              np.sin(self.perpendicular_angle) * .25 + self.origin_y ]
        self.p2 = [np.cos(self.perpendicular_angle) * -.25 + self.origin_x,
              np.sin(self.perpendicular_angle) * -.25 + self.origin_y ]

        self.K = [self.determine_K(self.p1), self.determine_K(self.p2)]

    def __str__(self):
        return f'a={self.a}, b={self.b}, offset={self.offset}, k={self.k}, grid={self.grid}'

    def determine_K(self, p):
        return np.ceil(self.a * p[0] + self.b * p[1] - self.offset)

    # Build function to get lines that hit boundaries
    # Maybe use build intersection function and use a line for each boundary!!!???
    def determine_intersection(self, other_line):

        denom = (self.a * other_line.b) - (other_line.a * self.b)
        if isclose(denom, 0, rel_tol=1e-5, abs_tol=1e-05):
            # Determine if it's the same line

            if isclose(other_line.a, 0, rel_tol=1e-5, abs_tol=1e-05):
                if not isclose(self.a, 0, rel_tol=1e-5, abs_tol=1e-05):
                    # Different lines
                    return None
                a_proportion = 0
            else:
                a_proportion = self.a / other_line.a
            
            if isclose(other_line.b, 0, rel_tol=1e-5, abs_tol=1e-05):
                if not isclose(self.b, 0, rel_tol=1e-5, abs_tol=1e-05):
                    # Different lines
                    return None
                b_proportion = 0
            else:
                b_proportion = self.b / other_line.b

            if a_proportion and b_proportion:
                if not isclose(a_proportion, b_proportion):
                    return None

            if isclose(other_line.c, 0, rel_tol=1e-5, abs_tol=1e-05):
                if not isclose(self.c, 0, rel_tol=1e-5, abs_tol=1e-05):
                    # Different lines
                    return None
                c_proportion = 0
            else:
                c_proportion = self.c / other_line.c

            if a_proportion and c_proportion:
                if not isclose(a_proportion, c_proportion, rel_tol=1e-5, abs_tol=1e-05):
                    return None
            
            if b_proportion and c_proportion:
                if not isclose(b_proportion, c_proportion, rel_tol=1e-5, abs_tol=1e-05):
                    return None

            raise OverlappingLines(self, other_line)
        else:
            x = (other_line.b * self.c) - (self.b * other_line.c)
            x = x / denom

            y = (self.a * other_line.c) - (other_line.a * self.c)
            y = y / denom

            return (x, y)
        
class OverlappingLines(Exception):
    def __init__(self, line_1, line_2, msg="These two lines are the same line."):
        self.line_1 = line_1
        self.line_2 = line_2
        self.msg = msg
        super().__init__(self.msg)

    def __str__(self):
        return f'{self.msg}\n\t{self.line_1}\n\t{self.line_2}'
    
class Tile:
    def __init__(self, points):
        self.points = points
        self.tile_group = 0

        # Don't need to compare distances between angles, always 1 by construction
        self.angles = []
        for i, point in enumerate(self.points):
            previous_point = self.points[(i - 1) % len(self.points)]
            next_point = self.points[(i + 1) % len(self.points)]

            in_vector = point - previous_point
            out_vector = next_point - point
            angle = np.arctan2(in_vector[0]*out_vector[1]-in_vector[1]*out_vector[0],
                            in_vector[0]*out_vector[0]+in_vector[1]*out_vector[1])

            self.angles.append(angle)

    def compare_angles(self, other_polygon):
        if len(self.angles) != len(other_polygon.angles):
            return False

        start = 0
        matched = False
        while start < len(self.angles) and not matched:
            for i in range(len(self.angles)):
                place = (i + start) % len(self.angles)
                if not isclose(self.angles[place], other_polygon.angles[i], rel_tol=1e-5, abs_tol=1e-05):
                    break
            if i == len(self.angles)-1:
                matched = True
            start += 1
        
        return matched
                
grids = []
for i, (s, c, o, bk) in enumerate(zip(sin_list, cos_list, offsets, base_k_values)):
    for k in np.arange(*grid_bounds):
        grids.append(Line(c, s, o, bk + k, i))

intersections = defaultdict(set)
for pair in combinations(grids, 2):
    l1 = pair[0]
    l2 = pair[1]
    
    if l1.grid == l2.grid:
        continue

    try:
        intersection = l1.determine_intersection(l2)
        if intersection:
            intersections[intersection].update([l1, l2])
    except OverlappingLines as e: 
        print(e)

base_tiles = []
tiles = []

for intersection_points, intersection_lines in intersections.items():

    # Maybe, get all the angles in order
    angles = list(intersection_lines)[0].angles.copy()
    angles.extend(list(intersection_lines)[1].angles.copy())
    angles.sort()
    angles
    # now determine which K goes on what side, (use offset for this?) maybe need to compute it, determine K by moving back 90 degrees
    # no, assign an appropriate K for each angle by moving counter-wise, pick K that is on the next side, probably need to work with quadrants
    # yeah, if angle1 is positive, then angle2 hits first, if angle1 is negative, then angle1 hit first
    # maybe??? if angle2 hits first, then k, else k+1

    # #start with right above (or below) the first one, (maybe need to get to second+ line to get all the K's) then continue looping through
    def determine_K(c, x, s, y, o):
        return np.ceil(c * x + s * y - o)

    ks = [determine_K(c, intersection_points[0], s, intersection_points[1], o) for c, s, o in zip(cos_list, sin_list, offsets)]

    # Now spin around to ensure the ks are correct on the lines that intersect
    for angle in angles:
        ks[angle.grid] = angle.k

    points = []
    for angle in angles:
        ks[angle.grid] = angle.k
        point = np.array([0, 0], dtype=float)
        for k, c, s in zip(ks, cos_list, sin_list):
            point += k * np.array([c, s])
        point -= mid_point # Center everything
        points.append(point)

    t = Tile(points)

    bt_match = False
    for bt in base_tiles:
        if bt.compare_angles(t):
            t.tile_group = bt.tile_group
            bt_match = True
            break

    if not bt_match:
        t.tile_group = len(base_tiles)
        base_tiles.append(t)

    tiles.append(t)

colors = ["blue", "green", "yellow", "orange", "red"]


1 15 29 0
1 8 15 0
1 4 8 0
4 6 8 4
6 7 8 6
6 6 7 6


In [ ]:
# boundary_max = abs(tiles[0].points[0][0])
# # x_total = 0
# # y_total = 0

# for t in tiles:
#     for p in t.points:
#         # x_total += p[0]
#         # y_total += p[1]

#         if abs(p[0]) > boundary_max:
#             boundary_max = abs(p[0])
#         if abs(p[1]) > boundary_max:
#             boundary_max = abs(p[1])

t_x_midpoint = tiles[0].points[0][0]
t_y_midpoint = tiles[0].points[0][1]
t_x_min = t_x_midpoint
t_y_min = t_y_midpoint
t_x_max = t_x_midpoint
t_y_max = t_y_midpoint
points_total = 0

for t in tiles:
    for p in t.points:
        t_x_midpoint += p[0]
        t_y_midpoint += p[1]

        if p[0] < t_x_min:
            t_x_min = p[0]
        if p[1] < t_y_min:
            t_y_min = p[1]
        if p[0] > t_x_max:
            t_x_max = p[0]
        if p[1] > t_y_max:
            t_y_max = p[1]

        points_total += 1

t_x_midpoint = t_x_midpoint / points_total
t_y_midpoint = t_y_midpoint / points_total

boundary_max = abs(t_x_min - t_x_midpoint)
if abs(t_x_max - t_x_midpoint) > boundary_max:
    boundary_max = abs(t_x_max - t_x_midpoint)
if abs(t_y_min - t_y_midpoint) > boundary_max:
    boundary_max = abs(t_y_min - t_y_midpoint)
if abs(t_y_max - t_y_midpoint) > boundary_max:
    boundary_max = abs(t_y_max - t_y_midpoint)

current_boundry = boundary_max
boundary_min = 1
no_white = True

step = 0
last_no_white = 0

while (boundary_max - boundary_min) > 1:
    
    fig, ax = plt.subplots(figsize=(5, 5))

    for t in tiles:
        p = Polygon(t.points, facecolor = colors[t.tile_group], edgecolor="k")
        ax.add_patch(p)
        
    ax.set_aspect('equal')
    # plt.axis([-current_boundry + t_x_midpoint, 
    #           current_boundry + t_x_midpoint, 
    #           -current_boundry + t_y_midpoint, 
    #           current_boundry + t_y_midpoint])
    plt.axis([-current_boundry, 
              current_boundry, 
              -current_boundry, 
              current_boundry])
    plt.axis('off')
    plt.tight_layout(pad=0)

    graph_file = f"output/graph.png"
    step += 1
    plt.savefig(graph_file, dpi=150)
    plt.close()

    im = Image.open(graph_file)

    no_white = True
    for pixel in im.getdata():
        if pixel == (255, 255, 255, 255):
            no_white = False
            break
    
    if no_white:
        last_no_white = current_boundry
        current_boundry, boundary_min = (current_boundry + boundary_max) / 2, current_boundry
    else:
        current_boundry, boundary_max = (current_boundry + boundary_min) / 2, current_boundry

    boundary_min = round(boundary_min)
    current_boundry = round(current_boundry)
    boundary_max = round(boundary_max)

    print(boundary_min, current_boundry, boundary_max, last_no_white)

fig, ax = plt.subplots(figsize=(5, 5))

for t in tiles:
    p = Polygon(t.points, facecolor = colors[t.tile_group], edgecolor="k")
    ax.add_patch(p)
    
ax.set_aspect('equal')
# plt.axis([-last_no_white + t_x_midpoint, 
#             last_no_white + t_x_midpoint, 
#             -last_no_white + t_y_midpoint, 
#             last_no_white + t_y_midpoint])
plt.axis([-last_no_white, 
            last_no_white, 
            -last_no_white, 
            last_no_white])
plt.axis('off')
plt.tight_layout(pad=0)

plt.savefig(graph_file, dpi=150)
plt.close()


In [140]:
last_no_white = 50

fig, ax = plt.subplots(figsize=(5, 5))

for t in tiles:
    p = Polygon(t.points, facecolor = colors[t.tile_group], edgecolor="k")
    ax.add_patch(p)
    
ax.set_aspect('equal')
plt.axis([-last_no_white, last_no_white, -last_no_white, last_no_white])
plt.tight_layout(pad=0)

graph_file = "output/graph.png"
plt.savefig(graph_file, dpi=150)
plt.close()

In [ ]:
# Try making the background transparent and pulling that instead of white
# remove tiles not touching other ones
# find border, set closest border point to be the diagonal length


-33.27050983124842


In [ ]:
edges = {}

for tile in tiles:
    

In [ ]:
tile = tiles[0]

edges = []

for p0, p1 in zip(tile.points, tile.points[1:] + tile.points[:1]):
    # left to right, then up to down
    if not isclose(p0[0], p1[0], rel_tol=1e-5, abs_tol=1e-05):
        if p0[0] < p1[0]:
            edges.append([])


Yeah, set up dictionary of points


[array([-6.66311896, -2.48989828]),
 array([-7.66311896, -2.48989828]),
 array([-7.97213595, -3.4409548 ]),
 array([-6.97213595, -3.4409548 ])]

In [145]:
tile.points

[array([-6.97213595, -3.4409548 ]),
 array([-6.66311896, -2.48989828]),
 array([-7.66311896, -2.48989828]),
 array([-7.97213595, -3.4409548 ])]